In [1]:
import ifcopenshell
import ifctester
from ifctester import reporter
from ifctester import ids

# Create the IDS Renhold (IDS Cleaning)

[IDS docutmentation](https://github.com/buildingSMART/IDS/blob/master/Documentation) in buildingSMART github repo. 


## Create IDS
Start by folling the [metadata guidelines found here.](https://github.com/buildingSMART/IDS/blob/master/Documentation/ids-metadata.md)

In [2]:
# create new IDS 
# ref https://github.com/buildingSMART/IDS/blob/master/Documentation/ids-metadata.md 
"""
my_ids = ids.Ids(
   title="IDS Renhold (IDS Cleaning)",
   author="sigve@mok-see.com",
   copyright="[mok-see] AS",
   version="0.5",
   purpose= "Basis for estimating costs of cleaning using IFC models",
   description="The Architectural IFC deliveries can be used to estimate costs of cleaning if it contains the \
      required information as defined in this IDS. This can be used by realestate owners to require deliveries from \
      architects and IFC models validated against this IDS can be provided as a basis for BIM based claning cost estimation.\
      IDS created as part of a pilot project headed by the Norwegian Building Authority",
   
   milestone="MMI 500: As-Built")
"""


'\nmy_ids = ids.Ids(\n   title="IDS Renhold (IDS Cleaning)",\n   author="sigve@mok-see.com",\n   copyright="[mok-see] AS",\n   version="0.5",\n   purpose= "Basis for estimating costs of cleaning using IFC models",\n   description="The Architectural IFC deliveries can be used to estimate costs of cleaning if it contains the       required information as defined in this IDS. This can be used by realestate owners to require deliveries from       architects and IFC models validated against this IDS can be provided as a basis for BIM based claning cost estimation.      IDS created as part of a pilot project headed by the Norwegian Building Authority",\n   \n   milestone="MMI 500: As-Built")\n'

In [3]:
# create new IDS 
# ref https://github.com/buildingSMART/IDS/blob/master/Documentation/ids-metadata.md 
my_ids = ids.Ids(
   title="IDS Renhold",
   author="sigve@mok-see.com",
   copyright="[mok-see] AS",
   version="0.5",
   purpose= "Basis for kalkulere renholdskostnader",
   description="Arkitektmodell kan bli brukt til å kalkulere renholdskostnader hvis den inneholder \
                informasjonen som er definert i denne Informasjons-leveranse-spesifikasjonen (IDS-Informaton Delivery Specification)\
                Denne IDS filen kan brukes for å automatisk validere at en IFC-modell. \
                Prosjektet er en del av pilotprosjekt i regi av DiBK",
   
   milestone="MMI 500 : Som bygget")

## Create specifications

## Requirement for naming
Rooms need to be given proper names and numbering that can be found on in the pphysical building

In [4]:
# Create naming specification
naming_spec = ids.Specification(
    name="Navngivning på rom",
    description= "Alle rom må ha et beskrivende navn og romnummer som kan gjennkjennes på merking i bygget",
    instructions="Alle rom må kunne identifiseres med romnavn og romnummer")
naming_spec.applicability.append(ids.Entity(name="IfcSpace"))

name_attr = ids.Attribute(
    name = "LongName",
    instructions="Alle rom må ha et romnavn",
    minOccurs=1
)
naming_spec.requirements.append(name_attr)

number_attr = ids.Attribute(
    name = "Name",
    instructions="Alle rom må ha et romnummer",
    minOccurs=1
)
naming_spec.requirements.append(number_attr)
# Add specification to ids.
my_ids.specifications.append(naming_spec)

In [5]:
covering_spec = ids.Specification(
    name="Dekkende materialer på rom",
    description= "Alle rom må referanse til dekkene som kler rommet",
    instructions="Alle rom må referanse til dekkene som kler rommet, som gir referanse til IfcCovering")
covering_spec.applicability.append(ids.Entity(name="IfcSpace"))

coverings = ids.Attribute(
    name = "HasCoverings",
    instructions="Alle rom må gi liste av relaterte dekkende flater",
    minOccurs=1
)
covering_spec.requirements.append(coverings)
my_ids.specifications.append(covering_spec)

## Requirement for room size
The size of the room is required in order to estimate cleaning costs

In [6]:

room_size_spec = ids.Specification(
    name="Størrelser på rom",
    description= "Romstørrelser må oppgis for å kunne kalkulere renholdskostnader",
    instructions="Alle rom må oppgi nødvendige størrelser for å kunne kalkulere riktige renholdskostnader per rom",
    ifcVersion = "IFC4")
room_size_spec.applicability.append(ids.Entity(name="IfcSpace"))

netFloorArea = ids.Property(
    name="NetFloorArea", 
    propertySet="Qto_SpaceBaseQuantities", 
    measure="IfcAreaMeasure",
    instructions="Netto gulvareal må være oppgitt per rom",
    minOccurs=1
    )
room_size_spec.requirements.append(netFloorArea)

finishCeilingHeight = ids.Property(
    name="FinishCeilingHeight", 
    propertySet="Qto_SpaceBaseQuantities", 
    measure="IfcLengthMeasure",
    instructions="Himlingshøyde, netto høyde på rommet, må være oppgitt for hver rom",
    minOccurs=1
    )
room_size_spec.requirements.append(finishCeilingHeight)

concealedCeiling = ids.Property(
    name="ConcealedCeiling", 
    propertySet="Qto_SpaceBaseQuantities", 
    measure="IfcLengthMeasure",
    instructions="Det må være oppgitt om det er åpen eller lukket himling på rommet",
    minOccurs=1
    )
room_size_spec.requirements.append(concealedCeiling)
my_ids.specifications.append(room_size_spec)



## Requirements for wals and floors and the realtion to rooms
Walls under 3 meter need to be cleaned, hence their net area and finish needs to be provided

In [7]:
# Vegger på opp til 3 meter skal oppgis nettoareal på
wall_spec = ids.Specification(
    name="Veggareal for vegger under 3 meter",
    description= "Alle vegger opp til og med 3 meter skal vaskes iht. INSTA800",
    instructions="Alle vegger opp til og med 3 meter må ha oppgitt nettoareal for å kunne kalkulere renholdskostnader",
    ifcVersion = "IFC4",
    minOccurs = 1
)

wall_spec.applicability.append(ids.Property(
    propertySet="Qto_WallBaseQuantities",
    name="Height",
    value="<=3")
    )

netWallArea = ids.Property(
    name="NetWallArea", 
    propertySet="Qto_SpaceBaseQuantities", 
    measure="IfcAreaMeasure",
    instructions="Alle vegger på 3 m eller mindre må ha oppgitt netto areal",
    minOccurs=1
    )
wall_spec.requirements.append(netWallArea)

wall_covering = ids.Attribute(
    name="HasCovering",
    instructions="Alle vegger under 3 meter skal ha referanse til materialene som dekker veggen",
    minOccurs=1
)
wall_spec.requirements.append(wall_covering)

my_ids.specifications.append(wall_spec)

my_ids.to_xml("renholds.ids")


True

In [8]:
# open  IFC file:
my_ifc = ifcopenshell.open("1170101-00-000-A-20_1st.ifc")

# validate IFC model against IDS requirements:
my_ids.validate(my_ifc)

# show results:
reporter.Json(my_ids).report()

{'title': 'IDS Renhold',
 'specifications': [{'name': 'Navngivning på rom',
   'status': True,
   'total_successes': 258,
   'total': 258,
   'percentage': 100,
   'required': False,
   'requirements': [{'description': 'The LongName shall be provided',
     'status': True,
     'failed_entities': []},
    {'description': 'The Name shall be provided',
     'status': True,
     'failed_entities': []}]},
  {'name': 'Dekkende materialer på rom',
   'status': False,
   'total_successes': 0,
   'total': 258,
   'percentage': 0,
   'required': False,
   'requirements': [{'description': 'The HasCoverings shall be provided',
     'status': False,
     'failed_entities': [{'reason': 'The attribute value "()" is empty',
       'element': "#361=IfcSpace('3COwjo3UT3exvXn8kT_eiS',#42,'08.401',$,$,#323,#356,'Sjakt 1',.ELEMENT.,.SPACE.,$)"},
      {'reason': 'The attribute value "()" is empty',
       'element': "#753=IfcSpace('3COwjo3UT3exvXn8kT_eiA',#42,'08.410',$,$,#729,#750,'Sjakt 2',.ELEMENT.,.SP

In [9]:
# open  IFC file:
my_ifc2 = ifcopenshell.open("KPV-A.ifc")

# validate IFC model against IDS requirements:
my_ids.validate(my_ifc2)

# show results:
reporter.Console(my_ids).report()

OSError: Unable to open file for reading

In [39]:
ids2 = ids.open("SampleIDS2.xml")

In [40]:
ids2.validate(my_ifc2)

In [41]:


reporter.Html(reporter.Html(ids2).report()).to_string()

'<!--\n    IfcTester - IDS based model auditing\n    Copyright (C) 2022 Dion Moult <dion@thinkmoult.com>\n\n    This file is part of IfcTester.\n\n    IfcTester is free software: you can redistribute it and/or modify\n    it under the terms of the GNU Lesser General Public License as published by\n    the Free Software Foundation, either version 3 of the License, or\n    (at your option) any later version.\n\n    IfcTester is distributed in the hope that it will be useful,\n    but WITHOUT ANY WARRANTY; without even the implied warranty of\n    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the\n    GNU Lesser General Public License for more details.\n\n    You should have received a copy of the GNU Lesser General Public License\n    along with IfcTester.  If not, see <http://www.gnu.org/licenses/>.\n-->\n<!DOCTYPE html>\n<html lang=>\n<head>\n    <meta charset="utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="description

In [42]:
from IPython.core.display import display, HTML


/tmp/ipykernel_392208/3492823427.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [43]:
engine = reporter.Html(ids2)
engine.report()

display(HTML(engine.to_string()))